### Day 9 - Part B
Finding the furthest points but also 

In [57]:
from util.aoc_utility import *

DAY = 9
TEST_MODE = 0
SECTIONS = False

data_in = load_input(day=DAY, test_mode=TEST_MODE, sections=SECTIONS)
data_sq = [line_to_arr(x, delimiter=",", convert_to_num=True) for x in data_in]

sqs = [(x[0], x[1]) for x in data_sq]

GRID_WIDTH = max([x[0] for x in sqs])
GRID_HEIGHT = max([x[1] for x in sqs])

In [58]:
def distance_to_corner(point, corner):
    """Get the distance from a given point to a corner of the grid
    
    Args:
        point (tuple): (x, y) coordinate in the grid
        corner (str): (0,0), (0,h), (w,0), (w,h) to indicate a grid corner 
        where w, h is the grid width and height

    Returns:
        dist (int): Numeric distance to the corner
    """
    #No need to sqrt since I care about the relative distance not the exact
    dist = (point[0] - corner[0])**2 + (point[1] - corner[1])**2
    return dist

def build_corner_maps(
        sqs, 
        corners=[(0,0), (0,GRID_HEIGHT), (GRID_WIDTH, 0), (GRID_WIDTH, GRID_HEIGHT)]
):
    """Function to build a list of distances between all points in sqs and the corners given
    
    Args:
        sqs (arr): Array of (x,y) points in the grid
        corners (arr): List of reference points (corners) in the grid

    Returns:
        corner_map (dict): Dictionary containing a sorted list of the closest points to each corner
    """

    corner_map = {}
    for corner in corners:

        dist_arr = []

        #Get the distance to the corner for each point
        for point in sqs:
            dist = distance_to_corner(point, corner)
            dist_arr.append((point, dist))

        dist_arr.sort(key=lambda x: x[1])

        corner_map[corner] = [x[0] for x in dist_arr]        

    return corner_map

In [59]:
def line_instersects(l1, l2):
    """For 2 given lines, do they cross each other (they can touch but not fully cross)
    Note in the case where l1 and l2 exist on the same axis, it is a breach if l1 exceeds l2
    
    Args:
        l1 (tuple): Tuple of coordinates of the form ((x,y), (x,y))
        l2 (tuple): Tuple of coordinates of the form ((x,y), (x,y))

    Returns:
        cross (bool): True if they cross each other
    """

    l1_xs = [x[0] for x in l1]
    l1_ys = [x[1] for x in l1]
    l2_xs = [x[0] for x in l2]
    l2_ys = [x[1] for x in l2]

    x_cross = not(max(l1_xs) <= min(l2_xs) or min(l1_xs) >= max(l2_xs))
    y_cross = not(max(l1_ys) <= min(l2_ys) or min(l1_ys) >= max(l2_ys))

    #Both lines along the same x axis
    if (l1_xs == l2_xs and l1_xs[0] == l1_xs[1]):
        if (l1_ys != l2_ys and l1_ys != l2_ys[::-1]) and y_cross:
            return True
        else:
            return False
        

    elif (l1_ys == l2_ys and l1_ys[0] == l1_ys[1]):
        if (l1_xs != l2_xs and l1_xs != l2_xs[::-1]) and x_cross:
            return True
        else:
            return False

    return x_cross and y_cross

def valid_rectangle(p1, p2, sqs):
    """For 2 given points, is it a valid rectangle in the grid
    
    Args:
        p1 (tuple): (x,y) point in the grid
        p2 (tuple): (x,y) point in the grid 
        sqs (arr): All points in the grid

    Returns:
        valid (bool): Whether the rectangle made from the points exist inside the points 
    """

    #Idea, get sides and if rectangle crosses (fully) one of the sides then not valid
    
    #Get the lines that make up the rectangle
    corners = [(p1[0], p1[1]), (p1[0], p2[1]), (p2[0],p2[1]), (p2[0], p2[1])]
    lines = [(corners[0], corners[1]), (corners[1], corners[2]), (corners[2], corners[3]), (corners[3], corners[0])]

    #For each line
    for line in lines:
        #Check against all lines made from sqs
        for idx, sq_1 in enumerate(sqs):
            if idx == len(sqs) - 1:
                sq_2 = sqs[0]
            else:
                sq_2 = sqs[idx+1]

            if line_instersects(line, (sq_1, sq_2)):
                return False
            
    return True

In [60]:
def get_rect_area(p1, p2):
    """Return the rectangle area created by two points
    
    Args:
        p1 (tuple): (x,y) point in the grid
        p2 (tuple): (x,y) point in the grid
        
    Returns:
        rect_area (int): Numeric value for the rectangle area.
    """

    return (abs(p1[0] - p2[0])+1) * (abs(p1[1] - p2[1])+1)

def biggest_rectangle_for_point(point, candidates):
    """Return the biggest rectangle possible for a given point using the corner map
    
    Args:
        point (tuple): (x,y) point in the grid
        candidates (arr): Array of candidate points for the biggest rectangle
        
    Returns:
        biggest_rect_area (int): Numeric value for the biggest rectangle
    """

    biggest_rect_area = 0
    for candidate in candidates:
        if valid_rectangle(point, candidate, candidates):
            candidate_rect_area = get_rect_area(point, candidate)

            if candidate_rect_area > biggest_rect_area:
                biggest_rect_area = candidate_rect_area

    return biggest_rect_area

In [ ]:
corner_map = build_corner_maps(sqs)

#Get all points closest and furthest from corners as candidates
candidates = set()
for corner in list(corner_map.keys()):
    candidates.add(corner_map[corner][0])
    candidates.add(corner_map[corner][-1])
candidates = list(candidates)

#Try all combinations for the biggest rectangle
best = (0, "")
for point in sqs:
    rect_area = biggest_rectangle_for_point(point, sqs)
    
    if rect_area > best[0]:
        best = (rect_area, point)

print(best)

(97633, 50169)
(97633, 51388)
(98053, 51388)
(98053, 52612)
(98121, 52612)
(98121, 53791)
(97553, 53791)
(97553, 55037)
(97842, 55037)
(97842, 56266)
(97846, 56266)
(97846, 57426)
(97350, 57426)
(97350, 58619)
(97125, 58619)
(97125, 59874)
(97196, 59874)
(97196, 61112)
(97124, 61112)
(97124, 62268)
(96696, 62268)
(96696, 63421)
(96283, 63421)
(96283, 64469)
(95550, 64469)
(95550, 65738)
(95513, 65738)
(95513, 66705)
(94617, 66705)
(94617, 67874)
(94289, 67874)
(94289, 68894)
(93601, 68894)
(93601, 70390)
(93964, 70390)
(93964, 71345)
(93126, 71345)
(93126, 72472)
(92651, 72472)
(92651, 73413)
(91831, 73413)
(91831, 74485)
(91258, 74485)
(91258, 75499)
(90587, 75499)
(90587, 76340)
(89659, 76340)
(89659, 77743)
(89559, 77743)
(89559, 78583)
(88636, 78583)
(88636, 79292)
(87566, 79292)
(87566, 80495)
(87129, 80495)
(87129, 81551)
(86489, 81551)
(86489, 82338)
(85538, 82338)
(85538, 82960)
(84428, 82960)
(84428, 83753)
(83513, 83753)
(83513, 84878)
(82919, 84878)
(82919, 85891)
(82195, 85